# A class for cleaning up Text and filling in proper nouns using neural coref

In [4]:
# !git clone https://github.com/huggingface/neuralcoref.git
# %cd neuralcoref
# !pip install -r requirements.txt
# !pip install -e .
# !python -m spacy download en_core_web_md

!pip install neuralcoref
!pip install bert-extractive-summarizer
!pip install spacy==2.1
!pip install transformers==2.2.2

     |████████████████████████████████| 27.7MB 131kB/s 
     |████████████████████████████████| 3.2MB 30.4MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 2.1MB 31.0MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
ERROR: en-core-web-md 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled preshed-3.0.2
  Found existing installation: thinc 7.3.1
    Uninstalling thinc-7.3.1:
      Successfully uninstalled thinc-7.3.1
  Found existing installation: spacy 2.2.2
    Uninstalling spacy-2.2.2:
      Successfully uninstalled spacy-2.2.2


In [5]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 95.4MB 55.7MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=6be3e6563216347199630e53ffa1d8bee7c9e187e00afd0ac8e57a957178ff4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-04j70018/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
  Found existing installation: en-core-web-md 2.2.5
    Uninstalling en-core-web-md-2.2.5:
      Successfully uninstalled en-core-web-md-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
# Sentence Handler
import neuralcoref
import spacy
from spacy.lang.en import English
from typing import List



class SentenceHandler(object):

    def __init__(self, language=English):
        self.nlp = language()
        self.nlp.add_pipe(self.nlp.create_pipe('sentencizer'))

    def process(self, body: str, min_length: int = 40, max_length: int = 600) -> List[str]:
        """
        Processes the content sentences.
        :param body: The raw string body to process
        :param min_length: Minimum length that the sentences must be
        :param max_length: Max length that the sentences mus fall under
        :return: Returns a list of sentences.
        """
        doc = self.nlp(body)
        return [c.string.strip() for c in doc.sents if max_length > len(c.string.strip()) > min_length]

    def __call__(self, body: str, min_length: int = 40, max_length: int = 600) -> List[str]:
        return self.process(body, min_length, max_length)


class CoreferenceHandler(SentenceHandler):

    def __init__(self, spacy_model: str = 'en_core_web_md', greedyness: float = 0.45):
        self.nlp = spacy.load(spacy_model)
        neuralcoref.add_to_pipe(self.nlp, greedyness=greedyness)

    def process(self, body: str, min_length: int = 40, max_length: int = 600):
        """
        Processes the content sentences.
        :param body: The raw string body to process
        :param min_length: Minimum length that the sentences must be
        :param max_length: Max length that the sentences mus fall under
        :return: Returns a list of sentences.
        """
        doc = self.nlp(body)._.coref_resolved
        doc = self.nlp(doc)
        return [c.string.strip() for c in doc.sents if max_length > len(c.string.strip()) > min_length]

# A class for clustering the embeddings

In [0]:
# Cluster Features
import numpy as np
from numpy import ndarray
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from typing import List


class ClusterFeatures(object):
    """
    Basic handling of clustering features.
    """

    def __init__(
        self,
        features: ndarray,
        algorithm: str = 'kmeans',
        pca_k: int = None,
        random_state: int = 12345
    ):
        """
        :param features: the embedding matrix created by bert parent
        :param algorithm: Which clustering algorithm to use
        :param pca_k: If you want the features to be ran through pca, this is the components number
        :param random_state: Random state
        """
        self.features = features

        self.algorithm = algorithm
        self.random_state = random_state
        np.random.seed(random_state)

    def __get_model(self, k: int):
        """
        Retrieve clustering model
        :param k: amount of clusters
        :return: Clustering model
        """

        if self.algorithm == 'gmm':
            return GaussianMixture(n_components=k, random_state=self.random_state)

        return KMeans(n_clusters=k, random_state=self.random_state)

    def __get_centroids(self, model):
        """
        Retrieve centroids of model
        :param model: Clustering model
        :return: Centroids
        """

        if self.algorithm == 'gmm':
            return model.means_
        return model.cluster_centers_

    def __find_closest_args(self, centroids: np.ndarray):
        """
        Find the closest arguments to centroid
        :param centroids: Centroids to find closest
        :return: Closest arguments
        """

        centroid_min = 1e10
        cur_arg = -1
        args = {}
        used_idx = []

        for j, centroid in enumerate(centroids):

            for i, feature in enumerate(self.features):
                value = np.linalg.norm(feature - centroid)

                if value < centroid_min and i not in used_idx:
                    cur_arg = i
                    centroid_min = value

            used_idx.append(cur_arg)
            args[j] = cur_arg
            centroid_min = 1e10
            cur_arg = -1

        return args

    def cluster(self, ratio: float = 0.1) -> List[int]:
        """
        Clusters sentences based on the ratio
        :param ratio: Ratio to use for clustering
        :return: Sentences index that qualify for summary
        """

        k = 1 if ratio * len(self.features) < 1 else int(len(self.features) * ratio)
        model = self.__get_model(k).fit(self.features)
        centroids = self.__get_centroids(model)
        cluster_args = self.__find_closest_args(centroids)
        sorted_values = sorted(cluster_args.values())
        return sorted_values

    def __call__(self, ratio: float = 0.1) -> List[int]:
        return self.cluster(ratio)

# Summarization code

In [0]:
body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

In [9]:
from transformers import *
import logging
import torch
import numpy as np

def cluster(body,ratio=0.3):
    # Sentences are cleaned up by the sentence handler
    nlp_sentence_handler = SentenceHandler()
    sentences = nlp_sentence_handler(body, min_length=2, max_length=600)
    
    # The Sentences are tokenised
    nlp_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
    tokenised_text_list = [ nlp_tokenizer.tokenize(t)              for t in sentences ]
    
    # Indexes of tokens are obtained
    indexed_tokens_list = [ nlp_tokenizer.convert_tokens_to_ids(t) for t in tokenised_text_list ]

    # Init the nlp model
    nlp_model = BertModel.from_pretrained('bert-large-uncased', output_hidden_states=True).to('cpu')
    nlp_model.eval()

    # The sentences are passes through the model one at a time because of variable length
    sentence_embeddings = []
    hidden = -2
    for x in indexed_tokens_list:
        sent_tensor = torch.tensor([x]).to('cuda:0')
        pooled, hidden_states = nlp_model(sent_tensor)[-2:]
        pooled = hidden_states[hidden].mean(dim=1)
        sentence_embeddings.append(pooled)

    hidden_features = np.array([np.squeeze(x.detach().cpu().numpy()) for x in sentence_embeddings])

    hidden_args = ClusterFeatures(hidden_features, algorithm='kmeans', random_state=12345).cluster(ratio)

    if hidden_args[0] != 0:
        hidden_args.insert(0,0)

    result = [sentences[j] for j in hidden_args]

    result = ''.join(result).replace('.','.\n')

    return result

print(cluster(body))

RuntimeError: ignored

In [0]:
game_of_thrones = '''
A Game of Thrones takes place over the course of one year on or near the fictional continent of Westeros. The story begins when King Robert visits the northern castle Winterfell to ask Ned Stark to be his right-hand assistant, or Hand of the King. The previous Hand, Jon Arryn, died under suspicious circumstances. Robert comes with his queen, Cersei Lannister, and his retinue, which includes a number of Lannisters. Just after the royal party arrives, Ned’s wife, Catelyn, receives a message claiming that the Lannister family was responsible for the death of the former Hand. She tells Ned, who accepts the position as Hand in order to protect Robert from the Lannisters. Ned’s son Bran then discovers Cersei Lannister and her brother Jaime Lannister having sex, and Jaime pushes Bran from a window to silence him. Everyone thinks Bran simply fell while climbing around the castle. While Bran is still unconscious, Ned leaves Winterfell and rides south with Robert. The same day, Ned’s bastard son, Jon, leaves to serve at the Wall, a massive structure that protects Westeros from the wilderness of the far North. The group of men sworn to defend the Wall, the Night’s Watch, have been receiving reports of strange creatures and have been losing men with increasing frequency. Tyrion Lannister, a little person who is brother to Cersei and Jaime, travels with Jon to the Wall to see the massive structure. Meanwhile, on a continent east of Westeros, Daenerys Targaryen marries the warlord Khal Drogo, one of the leaders of the Dothraki people. Daenerys and her brother Viserys are the last surviving members of the family Robert defeated to become king, the Targaryens. They are an old family said to be descended from dragons, and Viserys thinks with Khal Drogo’s army he can retake the throne. A knight named Ser Jorah Mormont, exiled by Ned Stark, pledges he will help. Daenerys receives three dragon eggs as a wedding gift and becomes immediately fascinated by them.
On the trip south to the capital, called King’s Landing, Robert’s and Cersei’s son, Joffrey, and Ned’s daughter Sansa, who everyone presumes will be married one day, go for a walk. When Joffrey sees Arya, Ned’s other daughter and sister to Sansa, practicing her swordfighting with a boy, he decides to show them he’s a better fighter. As pets, all of the Stark children have direwolf pups, a wolf breed larger than normal wolves that also happens to be the symbol of the Stark house, and Arya’s wolf injures Joffrey defending her. Though Sansa knows Joffrey instigated the fight, she will not tell on Joffrey because she’s in love with him. As punishment, Cersei wants Arya’s wolf killed, but since it ran away after hurting Joffrey, Cersei demands that Ned kill Sansa’s wolf instead. Meanwhile, an assassin tries to kill the unconscious Bran and fails. Ned finally reaches King’s Landing to find that Catelyn has sailed to the city in secret to discover the truth about the assassin. She has the dagger the assassin used, and after examining it, Catelyn’s childhood friend Littlefinger recognizes it as belonging to Tyrion Lannister. Ned tells Catelyn he will try to determine who killed the former Hand, Jon Arryn, and tried to kill Bran. Bran finally wakes from his coma, but he doesn’t remember how he fell. Tyrion visits him on his way south from the Wall to deliver a greeting from Jon. Tyrion continues south as Catelyn starts back north, and when their paths cross Catelyn has him seized for trying to kill Bran.
In King’s Landing, Ned slowly begins to unravel the mystery of why the previous Hand was killed. He knows it has to do with something the Hand learned about King Robert’s children. Through a spy, Robert learns that Daenerys Targaryen is pregnant. He wants to assassinate her because he fears she and her son will one day challenge Robert’s right to the throne. Disgusted with Robert’s plan, Ned resigns as Hand. That night, Jaime and his men confront Ned about Tyrion’s capture. Jaime kills Ned’s men and Ned breaks his leg while fighting. The following day, Robert reinstates Ned as hand. While Robert is gone hunting, Ned orders the execution of a rogue knight loyal to the Lannister family who has been pillaging villages. Further north, Catelyn takes Tyrion to her sister Lysa Arryn’s castle, the Eyrie, which is in a mountainous area called The Vale. Lysa accuses Tyrion of arranging the murder of both Jon Arryn and Bran. Tyrion denies the accusations and demands a trial by combat. A knight fights on Lysa’s behalf, and a mercenary fights on Tyrion’s behalf. Tyrion’s mercenary wins. While Tyrion rides from the Eyrie, a group of mountain clansmen try to kill him, but he promises to help them take The Vale and convinces them to join him.
In the east, as Khal Drogo and his Dothraki followers head back to Vaes Dothrak, their capital, Viserys becomes increasingly angry that Drogo has not provided him with an army with which to wage war on Westeros. During a feast he attacks Daenerys in a rage, and Khal Drogo has Viserys killed by dumping molten gold on his head. In King’s Landing, Ned has figured out why the Hand was killed: he had discovered that Joffrey was not really Robert’s child but was actually the product of Cersei’s sexual relationship with her brother, Jaime. Robert doesn’t know the truth. Robert is mortally wounded in a hunt, and before he dies, he names Ned the Protector of the Realm, essentially an interim king, until Joffrey comes of age. Ned does not tell Robert that he knows Joffrey is not the true heir, since he is the son of Cersei and Jaime. Ned asks Littlefinger’s help to install the true heir, Robert’s brother Stannis, as the king, and Littlefinger agrees. But when Ned confronts the Lannisters, saying that Joffrey is not the true heir and expecting Littlefinger’s support, Littlefinger betrays him, and Cersei imprisons Ned for treason. Meanwhile, north of the Wall, Jon and other men have discovered two strange dead bodies. They bring the bodies back for examination, and late at night, one of the bodies comes to life and tries to kill Jon’s commander. Jon and his direwolf fight off the undead body, and Jon kills the creature with fire.
After Ned’s capture, Arya escapes the castle in King’s Landing and Cersei holds Sansa hostage (she says she is holding Sansa for her own protection). Tywin Lannister, father to Tyrion, Cersei, and Jaime, wages war with Catelyn and her son, Robb Stark. Shrewdly outmaneuvering Tywin, Robb manages to defeat a portion of the Lannister army and capture Jaime. In King’s Landing, Joffrey, who is considered Robert’s heir, is crowned King. In the hopes that he can prevent his daughters being harmed, Ned confesses publicly to treason, and Joffrey has him executed while Sansa watches. Arya is in the crowd, though nobody knows it. Learning of his father’s death and his brother’s march to war, Jon tries to desert the Wall. But Jon’s friends convince him that he must stay and defend the Wall as he vowed. In the east, Drogo suffers a wound while raiding a village. Daenerys has one of the village women treat him, but he becomes very sick. When he is about to die, Daenerys asks the woman to heal him, but she says only bloodmagic will save him and it will require a death in exchange for his life. Despite the protests of the Dothraki people, Daenerys has the woman do it. Shortly thereafter Daenerys goes into labor. When she wakes many days later, her child is dead and Drogo is alive but brain dead. His followers have all left, leaving behind only a few warriors, the sick and old, and Ser Jorah Mormont, who has become one of Daenerys’s most trusted advisors. Daenerys sets a funeral pyre to burn Drogo and the woman, who tricked her and essentially killed her husband and child. Daenerys also places the three dragon eggs into the pyre. As the fire burns, Daenerys walks into it, and when finally it clears, Mormont and the Dothrakis find her with three newborn dragons at her breast.
'''
print(cluster(game_of_thrones))